# Hızlı Başlangıç

Bu defteri Colab'da güvenle çalıştırmak için özet adımlar:

- GPU: Runtime → Change runtime type → GPU seçin.
- Sıra: 1) Drive mount, 2) Kütüphaneler, 3) Config, 4) Dataset/DataLoader, 5) Model, 6) Train, 7) Eval, 8) (Opsiyonel) Görselleştirme.
- Config (Cell 5):
  - `OUT_ROOT`/`IMG_ROOT`: TMN çıktıları (ds2_dense_tmn) klasörlerinize işaret etmeli.
  - `BATCH_SIZE`: Küçük başlayın (2), A100’de stabil ise artırın.
  - `EVAL_SPLIT`: 'train' veya 'test' (mAP raporu bu split’e göre yazılır).
  - `MAX_INSTANCES`: `None` = sınırsız; RAM kısıtlıysa makul bir sayı verin.
  - `DATA_LOADER_PIN_MEMORY`: Colab RAM baskısını azaltmak için varsayılan False.
- Çıktılar: `RUN_DIR` otomatik oluşturulur (`maskrcnn/YYYYMMDD_HHMM`).
  - `logs/` (eğitim günlüğü), `checkpoints/` (ağırlıklar), `reports/` (mAP ve tespitler) burada.
- Değerlendirme: `EVAL_SPLIT`’e göre mAP yazılır: `reports/metrics_<split>.json`.
- İpuçları: `num_workers=0`, `pin_memory=False` Colab stabilitesine yardımcı olur.


In [ ]:
# Colab: Mount Google Drive
import sys, os
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive mounted at /content/drive')
else:
    print('Not in Colab; skipping drive mount.')

# TMN Mask R-CNN Training (Dedicated Notebook)
This notebook focuses only on training to avoid mixing with augmentation tasks. It sets up GPU, loads DS2 Dense TMN from Drive, trains Mask R-CNN, and saves checkpoints.

In [ ]:
# Proje Yapılandırması ve Kütüphanelerin İçe Aktarılması
import os, sys, json, glob, time
import torch, torchvision
from PIL import Image
from pathlib import Path
print('GPU:', torch.cuda.is_available(), torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU')

In [ ]:
# Bölümler

Bu defter şu bölümlerden oluşur:
- preprocess: veri yolları ve dataloader hazırlığı
- train: model kurulumu ve eğitim
- eval: görselleştirme ve mAP değerlendirme
- logs: eğitim günlükleri ve checkpoint konumu


In [ ]:
# Parametreleştirme: Config Sözlüğü ve Dataclass
from dataclasses import dataclass
from typing import Optional

@dataclass
class Config:
    # Eğitim girdileri: TMN dataset
    OUT_ROOT: str = '/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_dense_tmn'
    IMG_ROOT: str = '/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_dense_tmn/images'
    # Eğitim çıktıları: ayrı bir train klasöründe kalıcı
    TRAIN_ROOT: str = '/content/drive/MyDrive/omr_dataset/dataset/ds2/train/ds2_dense_tmn'
    # Model özel alt klasörü
    MASKRCNN_ROOT: str = '/content/drive/MyDrive/omr_dataset/dataset/ds2/train/ds2_dense_tmn/maskrcnn'
    # Bellek için daha konservatif başlangıç batch boyutu
    BATCH_SIZE: int = 2
    EPOCHS: int = 3
    LR: float = 0.005
    MOMENTUM: float = 0.9
    WEIGHT_DECAY: float = 0.0005
    # Bellek kontrolleri
    MAX_INSTANCES: Optional[int] = None   # Görsel başına sınır yok (RAM yeterliyse)
    DATA_LOADER_PIN_MEMORY: bool = False  # RAM tüketimini azaltmak için kapalı
    # Değerlendirme ayarları
    EVAL_SPLIT: str = 'test'   # 'test' yoksa 'train' olarak ayarla
    EVAL_SCORE_THR: float = 0.05

cfg = Config()
print(cfg)

# Çalışma klasörü: tarih/saat etiketli alt klasör (maskrcnn altında)
import time, os
os.makedirs(cfg.MASKRCNN_ROOT, exist_ok=True)
RUN_DIR = os.path.join(cfg.MASKRCNN_ROOT, time.strftime('%Y%m%d_%H%M'))
os.makedirs(RUN_DIR, exist_ok=True)
print('RUN_DIR:', RUN_DIR)

In [ ]:
# Kategori Haritası: category_id -> eğitim etiketi (1..K), 0 arkaplan
import json, glob, os

train_jsons = sorted(glob.glob(f"{cfg.OUT_ROOT}/jsonlar/*train*.json"))
test_jsons = sorted(glob.glob(f"{cfg.OUT_ROOT}/jsonlar/*test*.json"))

def build_category_maps(json_paths):
    cats = set()
    for jp in json_paths:
        try:
            with open(jp, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except Exception:
            continue
        anns = data.get('annotations') or {}
        ann_iter = anns.values() if isinstance(anns, dict) else anns
        for a in ann_iter:
            cats_val = a.get('cat_id') or a.get('category_id')
            if isinstance(cats_val, list):
                for c in cats_val:
                    try:
                        cats.add(int(c))
                    except Exception:
                        pass
            elif cats_val is not None:
                try:
                    cats.add(int(cats_val))
                except Exception:
                    pass
    cats = sorted(cats)
    cat_map = {orig: i+1 for i, orig in enumerate(cats)}  # 0: background
    inv_cat_map = {v: k for k, v in cat_map.items()}
    return cat_map, inv_cat_map

ALL_JSONS = train_jsons + test_jsons
CAT_MAP, INV_CAT_MAP = build_category_maps(ALL_JSONS)
NUM_CLASSES = 1 + len(CAT_MAP)
print({'num_classes': NUM_CLASSES, 'categories': len(CAT_MAP)})


In [ ]:
# Kategori Özeti: ID ve (varsa) isimleri yazdır
import json

def _collect_category_names(json_paths):
    names = {}
    for jp in json_paths:
        try:
            with open(jp, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except Exception:
            continue
        cats = data.get('categories') or []
        if isinstance(cats, dict):
            cats = list(cats.values())
        for c in cats:
            try:
                cid = int(c.get('id'))
            except Exception:
                continue
            nm = c.get('name') or c.get('category_name') or None
            if cid not in names and nm:
                names[cid] = str(nm)
    return names

NAME_BY_CAT_ID = _collect_category_names(ALL_JSONS)

ORIG_CATEGORY_IDS = sorted(CAT_MAP.keys())
TRAIN_LABEL_TO_CATEGORY = {tr: INV_CAT_MAP[tr] for tr in sorted(INV_CAT_MAP.keys())}

print('Orijinal category_id listesi (sıralı):', ORIG_CATEGORY_IDS)
print('Toplam kategori:', len(ORIG_CATEGORY_IDS))
print('\nEğitim label -> category_id (ve ad):')
for tr in range(1, NUM_CLASSES):
    cid = TRAIN_LABEL_TO_CATEGORY.get(tr)
    nm = NAME_BY_CAT_ID.get(cid)
    if nm is not None:
        print(f'  {tr:2d} -> {cid}  ({nm})')
    else:
        print(f'  {tr:2d} -> {cid}')

# Opsiyonel: dizi olarak sakla
TRAIN_CATEGORY_IDS = [TRAIN_LABEL_TO_CATEGORY[i] for i in range(1, NUM_CLASSES)]
TRAIN_CATEGORY_NAMES = [NAME_BY_CAT_ID.get(cid) for cid in TRAIN_CATEGORY_IDS]
print('\nTRAIN_CATEGORY_IDS:', TRAIN_CATEGORY_IDS)
print('TRAIN_CATEGORY_NAMES:', TRAIN_CATEGORY_NAMES)


In [ ]:
# Günlükleme (Logging)
import logging, os
LOG_DIR = os.path.join(RUN_DIR, 'logs')
os.makedirs(LOG_DIR, exist_ok=True)
logging.basicConfig(level=logging.INFO, handlers=[
    logging.FileHandler(os.path.join(LOG_DIR, 'train.log')),
    logging.StreamHandler()
])
logging.info('Logging initialized at %s', LOG_DIR)

In [ ]:
# Veri İşleme Akışı ve Dataset
import torch
from torch.utils.data import Dataset, DataLoader

class DS2TMNDataset(Dataset):
    def __init__(self, images_dir, json_paths, transform=None, max_instances=None, category_map=None):
        import json
        self.images_dir = images_dir
        self.transform = transform
        self.max_instances = max_instances
        self.category_map = category_map
        self.items = []  # {'filename': str, 'image_id': int}
        self.anns_by_fn = {}  # filename -> List[[x1,y1,x2,y2,label]]

        seen_fn = set()
        for jp in json_paths:
            with open(jp, 'r', encoding='utf-8') as f:
                data = json.load(f)

            imgs = data.get('images') or []
            if isinstance(imgs, dict):
                imgs = list(imgs.values())
            id_to_fn = {}
            for im in imgs:
                fn = im.get('filename') or im.get('file_name')
                if not fn:
                    continue
                try:
                    iid = int(im.get('id')) if im.get('id') is not None else None
                except Exception:
                    iid = None
                if iid is not None:
                    id_to_fn[iid] = fn

            anns = data.get('annotations') or {}
            ann_iter = anns.values() if isinstance(anns, dict) else anns
            for a in ann_iter:
                img_id = a.get('img_id') or a.get('image_id')
                if img_id is None:
                    continue
                try:
                    fn = id_to_fn[int(img_id)]
                except Exception:
                    continue
                b = a.get('a_bbox') or a.get('bbox')
                cats = a.get('cat_id') or a.get('category_id') or []
                if not b or len(b) < 4:
                    continue
                # Etiket çözümleme (liste/tekil)
                if isinstance(cats, list) and len(cats) > 0:
                    orig_lab = int(cats[0])
                elif isinstance(cats, (int, str)):
                    orig_lab = int(cats)
                else:
                    orig_lab = 0
                # Eğitim için map'lenmiş etiket (1..K), 0 arkaplan kullanılmaz
                if self.category_map is not None:
                    mapped = self.category_map.get(orig_lab)
                    if mapped is None:
                        # bilinmeyense atla
                        continue
                    lab_to_use = int(mapped)
                else:
                    lab_to_use = int(orig_lab)
                self.anns_by_fn.setdefault(fn, []).append([
                    float(b[0]), float(b[1]), float(b[2]), float(b[3]), lab_to_use
])

            # Bu shard'daki görselleri ekle (fn bazlı tekil)
            for iid, fn in id_to_fn.items():
                if fn in seen_fn:
                    continue
                seen_fn.add(fn)
                # image_id'i bu shard'dan alıyoruz; test/train ayrıldığı için mAP eşleşmesi korunur
                self.items.append({'filename': fn, 'image_id': int(iid) if iid is not None else -1})

        # Görselleri alfabetik sırala (tekrar üretilebilirlik için)
        self.items.sort(key=lambda x: x['filename'])
        self.to_tensor = torchvision.transforms.ToTensor()

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        im = self.items[idx]
        fn = im['filename']
        img_id = int(im['image_id'])
        path = os.path.join(self.images_dir, fn)
        try:
            img = Image.open(path).convert('RGB')
        except Exception:
            W = H = 1
            img = Image.new('RGB', (W, H), (0, 0, 0))
            lst = []
        else:
            W, H = img.size
            lst = self.anns_by_fn.get(fn, [])

        boxes_labels = []
        for rec in lst:
            x1, y1, x2, y2, lab = rec
            x1 = max(0, min(x1, W - 1)); y1 = max(0, min(y1, H - 1))
            x2 = max(0, min(x2, W));     y2 = max(0, min(y2, H))
            if x2 > x1 and y2 > y1:
                boxes_labels.append((x1, y1, x2, y2, lab))

        # Instance sınırı (bellek için)
        if self.max_instances and len(boxes_labels) > self.max_instances:
            boxes_labels = boxes_labels[: self.max_instances]

        boxes = [[x1, y1, x2, y2] for x1, y1, x2, y2, _ in boxes_labels]
        labels = [lab for *_, lab in boxes_labels]

        if len(boxes) > 0:
            masks = torch.zeros((len(boxes), H, W), dtype=torch.bool)
            for i, (x1, y1, x2, y2) in enumerate(boxes):
                masks[i, int(y1):int(y2), int(x1):int(x2)] = True
        else:
            masks = torch.zeros((0, H, W), dtype=torch.bool)

        target = {
            'boxes': torch.tensor(boxes, dtype=torch.float32),
            'labels': torch.tensor(labels, dtype=torch.int64),
            'masks': masks,
            'image_id': torch.tensor([img_id], dtype=torch.int64)
        }
        img = self.transform(img) if self.transform else self.to_tensor(img)
        return img, target

train_jsons = sorted(glob.glob(f"{cfg.OUT_ROOT}/jsonlar/*train*.json"))
test_jsons = sorted(glob.glob(f"{cfg.OUT_ROOT}/jsonlar/*test*.json"))
train_ds = DS2TMNDataset(images_dir=cfg.IMG_ROOT, json_paths=train_jsons, max_instances=cfg.MAX_INSTANCES, category_map=CAT_MAP)
test_ds = DS2TMNDataset(images_dir=cfg.IMG_ROOT, json_paths=test_jsons, max_instances=cfg.MAX_INSTANCES, category_map=CAT_MAP)

def collate_fn(batch):
    return tuple(zip(*batch))

# RAM tüketimini kontrol altında tutmak için otomatik büyütmeyi kaldırdık
bs = cfg.BATCH_SIZE
# Colab'de worker öldürme sorunlarını önlemek için num_workers=0; pin_memory bellek için kapalı
train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=0, pin_memory=cfg.DATA_LOADER_PIN_MEMORY, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=bs, shuffle=False, num_workers=0, pin_memory=cfg.DATA_LOADER_PIN_MEMORY, collate_fn=collate_fn)
print({'batch_size': bs, 'train_len': len(train_ds), 'test_len': len(test_ds)})

# Etiket aralığı hızlı kontrol (örneklem)
def _check_label_ranges(ds, name, max_samples=100):
    seen_min, seen_max, checked, bad = None, None, 0, 0
    for i in range(min(max_samples, len(ds))):
        _, t = ds[i]
        l = t['labels']
        if l.numel() == 0:
            continue
        lmin = int(l.min())
        lmax = int(l.max())
        seen_min = lmin if seen_min is None else min(seen_min, lmin)
        seen_max = lmax if seen_max is None else max(seen_max, lmax)
        if (l < 1).any() or (l > (NUM_CLASSES-1)).any():
            bad += 1
        checked += 1
    print({name: {'sample_min': seen_min, 'sample_max': seen_max, 'checked': checked, 'out_of_range_samples': bad}})

_check_label_ranges(train_ds, 'train_labels')

In [ ]:
# Modülerleştirme ve Model Kurulumu
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

num_classes = NUM_CLASSES  # background(0) + K kategori (1..K)
model = maskrcnn_resnet50_fpn(weights=None)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, 256, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print('Model on', device, '| num_classes =', num_classes)


In [ ]:
# Eğitim Döngüsü (AMP) ve Checkpoint
from torch.optim import SGD
from torch.amp import GradScaler, autocast
import os, time, gc

optimizer = SGD(model.parameters(), lr=cfg.LR, momentum=cfg.MOMENTUM, weight_decay=cfg.WEIGHT_DECAY)
scaler = GradScaler('cuda', enabled=torch.cuda.is_available())
CKPT_DIR = os.path.join(RUN_DIR, 'checkpoints')
os.makedirs(CKPT_DIR, exist_ok=True)

model.train()
for epoch in range(cfg.EPOCHS):
    t0 = time.time(); total = 0.0
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) if torch.is_tensor(v) else v for k,v in t.items()} for t in targets]
        optimizer.zero_grad(set_to_none=True)
        with autocast('cuda', enabled=torch.cuda.is_available()):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
        scaler.scale(losses).backward()
        scaler.step(optimizer)
        scaler.update()
        total += losses.item()
        # Batch sonu temizlik (özellikle RAM/VRAM baskısını azaltmak için)
        del images, targets, loss_dict, losses
    dur = time.time()-t0
    avg = total / max(1,len(train_loader))
    logging.info({'epoch': epoch+1, 'loss': round(avg,3), 'sec': round(dur,1)})
    ckpt = os.path.join(CKPT_DIR, f'maskrcnn_epoch{epoch+1}.pt')
    torch.save({'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch+1}, ckpt)
    logging.info({'saved': ckpt})
    # Epoch sonu bellek temizliği
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
print('Done')

In [ ]:
# Değerlendirme ve Görselleştirme
import matplotlib.pyplot as plt
model.eval()
@torch.no_grad()
def eval_show(n=3, thr=0.5):
    shown = 0
    for images, targets in test_loader:
        images = [img.to(device) for img in images]
        outputs = model(images)
        for img, out in zip(images, outputs):
            if shown>=n: return
            fig, ax = plt.subplots(figsize=(6,6))
            ax.imshow(img.permute(1,2,0).cpu().numpy())
            boxes = out['boxes'].cpu().numpy(); scores = out['scores'].cpu().numpy()
            for b,s in zip(boxes, scores):
                if s<thr: continue
                x1,y1,x2,y2 = b
                ax.add_patch(plt.Rectangle((x1,y1), x2-x1, y2-y1, fill=False, color='y', linewidth=2))
            ax.set_title(f'detections >= {thr}')
            plt.show(); plt.close(fig)  # Bellek sızıntılarını önlemek için figürü kapat
            shown += 1

eval_show(3, 0.5)

In [ ]:
# COCO mAP Evaluation (train/test sabit eşleme, RAM dostu akış)
import json, os, gc
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Split seçimi: 'train' -> train_jsons & train_loader, 'test' -> test_jsons & test_loader
split = (cfg.EVAL_SPLIT or 'test').lower()
if split == 'test':
    assert len(test_jsons) > 0, "test split için JSON bulunamadı (ör. deepscores_test.json)"
    EVAL_JSON = test_jsons[0]
    eval_loader = test_loader
elif split == 'train':
    assert len(train_jsons) > 0, "train split için JSON bulunamadı (ör. deepscores_train.json)"
    EVAL_JSON = train_jsons[0]
    eval_loader = train_loader
else:
    raise ValueError(f"Bilinmeyen EVAL_SPLIT: {split}. 'train' veya 'test' olmalı.")

print({'eval_split': split, 'json': EVAL_JSON})
cocoGt = COCO(EVAL_JSON)

# Akış tabanlı tespit toplama: JSONL dosyasına yaz, bellekte tutma
REPORT_DIR = os.path.join(RUN_DIR, 'reports')
os.makedirs(REPORT_DIR, exist_ok=True)
DETS_JSONL = os.path.join(REPORT_DIR, f'detections_{split}.jsonl')
DETS_JSON = os.path.join(REPORT_DIR, f'detections_{split}.json')  # COCO loadRes JSON array ister

model.eval()
import numpy as np
@torch.no_grad()
def collect_detections_stream(score_thr=0.05):
    # Var olan dosyayı temizle
    if os.path.exists(DETS_JSONL):
        os.remove(DETS_JSONL)
    processed = 0
    with open(DETS_JSONL, 'w', encoding='utf-8') as fw:
        for images, targets in eval_loader:
            images = [img.to(device) for img in images]
            outputs = model(images)
            for out, tgt in zip(outputs, targets):
                img_id = int(tgt['image_id'].item())
                boxes = out['boxes'].detach().cpu().numpy()
                scores = out['scores'].detach().cpu().numpy()
                labels = out['labels'].detach().cpu().numpy()
                for b, s, lab in zip(boxes, scores, labels):
                    if s < score_thr:
                        continue
                    x1,y1,x2,y2 = b
                    # Eğitim label'ını COCO GT'deki orijinal category_id'ye geri çevir
                    orig_cat = int(INV_CAT_MAP.get(int(lab), int(lab)))
                    rec = {
                        'image_id': img_id,
                        'category_id': orig_cat,
                        'bbox': [float(x1), float(y1), float(x2-x1), float(y2-y1)],
                        'score': float(s)
                    }
                    fw.write(json.dumps(rec, ensure_ascii=False) + '\n')
            # Bellek temizliği (batch bazlı)
            del images, outputs
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()
            processed += len(targets)
    return processed

# Tüm seçilen split'i işle (RAM şişmeden)
processed = collect_detections_stream(score_thr=cfg.EVAL_SCORE_THR)
print(f'Processed images ({split}):', processed)

# JSONL -> JSON Array (stream ederek)
with open(DETS_JSONL, 'r', encoding='utf-8') as fr, open(DETS_JSON, 'w', encoding='utf-8') as fw:
    fw.write('[')
    first = True
    for line in fr:
        line = line.strip()
        if not line:
            continue
        if not first:
            fw.write(',')
        fw.write(line)
        first = False
    fw.write(']')

# Evaluate mAP
# COCO loadRes dosya yolunu kabul eder (JSON array formatında)
try:
    cocoDt = cocoGt.loadRes(DETS_JSON)
except Exception as e:
    print('Failed to load detections for COCOeval:', e)
    cocoDt = None

if cocoDt is None:
    print('No detections to evaluate or load failed.')
else:
    cocoEval = COCOeval(cocoGt, cocoDt, iouType='bbox')
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()
    # Save metrics
    metrics = {
        'AP@[.5:.95]': float(cocoEval.stats[0]),
        'AP@0.5': float(cocoEval.stats[1]),
        'AP@0.75': float(cocoEval.stats[2])
    }
    with open(os.path.join(REPORT_DIR, f'metrics_{split}.json'), 'w', encoding='utf-8') as f:
        json.dump(metrics, f)
    print('Saved metrics to', os.path.join(REPORT_DIR, f'metrics_{split}.json'))
    # Bellek temizliği
    del cocoDt, cocoEval
    gc.collect()

In [ ]:
# Çıktı Özeti (RUN_DIR, checkpoint ve raporlar)
import os, glob, json

if 'RUN_DIR' not in globals():
    print('RUN_DIR tanımlı değil. Önce Config hücresini çalıştırın.')
else:
    print('RUN_DIR          :', RUN_DIR)
    ckpt_dir = os.path.join(RUN_DIR, 'checkpoints')
    rep_dir  = os.path.join(RUN_DIR, 'reports')

    print('\n[checkpoints]')
    ckpts = sorted(glob.glob(os.path.join(ckpt_dir, '*.pt')))
    print('adet:', len(ckpts))
    if ckpts:
        print('son:', ckpts[-1])

    print('\n[reports]')
    if os.path.isdir(rep_dir):
        reps = sorted(glob.glob(os.path.join(rep_dir, '*')))
        for p in reps[:20]:
            print('-', os.path.basename(p))
        # metrics_<split>.json varsa göster
        for split_name in ['train', 'test']:
            mpath = os.path.join(rep_dir, f'metrics_{split_name}.json')
            if os.path.exists(mpath):
                try:
                    with open(mpath, 'r', encoding='utf-8') as f:
                        metrics = json.load(f)
                    print(f"\nmetrics_{split_name}.json:", metrics)
                except Exception as e:
                    print(f"metrics_{split_name}.json okunamadı:", e)
    else:
        print('Rapor klasörü yok: önce değerlendirme hücresini çalıştırın.')
